In [1]:
from pyiem.util import get_dbconn
ASOS = get_dbconn('asos')
cursor = ASOS.cursor()

cursor.execute("""
 select distinct date(valid) from alldata where station = 'OKC' 
 and array_to_string(wxcodes, '') LIKE '%%FZDZ%%' and
 extract(hour from ((valid + '10 minutes'::interval) at time zone 'UTC')) in (21, 22, 23, 0, 1, 2, 3, 4)
 and extract(month from valid) in (11, 12,1,2, 3, 4) ORDER by date ASC
""")
wdates = []
for row in cursor:
    wdates.append( row[0] )
    
ASOS.close()
print(wdates)

[datetime.date(1948, 1, 26), datetime.date(1948, 2, 5), datetime.date(1948, 2, 12), datetime.date(1949, 1, 9), datetime.date(1949, 1, 10), datetime.date(1949, 1, 11), datetime.date(1949, 1, 17), datetime.date(1949, 1, 25), datetime.date(1949, 1, 26), datetime.date(1949, 2, 2), datetime.date(1949, 2, 13), datetime.date(1949, 2, 20), datetime.date(1949, 12, 25), datetime.date(1951, 2, 14), datetime.date(1951, 11, 5), datetime.date(1952, 1, 3), datetime.date(1952, 11, 28), datetime.date(1953, 1, 22), datetime.date(1953, 3, 3), datetime.date(1954, 1, 13), datetime.date(1954, 12, 27), datetime.date(1956, 1, 22), datetime.date(1956, 1, 29), datetime.date(1956, 2, 1), datetime.date(1956, 2, 3), datetime.date(1956, 2, 4), datetime.date(1956, 12, 6), datetime.date(1956, 12, 13), datetime.date(1956, 12, 18), datetime.date(1957, 1, 26), datetime.date(1957, 1, 27), datetime.date(1957, 1, 30), datetime.date(1957, 2, 22), datetime.date(1957, 2, 23), datetime.date(1958, 2, 9), datetime.date(1958, 12,

In [ ]:
POSTGIS = get_dbconn('postgis')
cursor = POSTGIS.cursor()

cursor.execute("""
 select f.valid, p.smps, p.height from raob_profile p JOIN raob_flights f on (p.fid = f.fid) 
 WHERE f.station in ('KOAX','KOMA','KOVN') and p.smps * 2.23 > 30 and p.levelcode = 9 
 and extract(hour from f.valid at time zone 'UTC') = 0 ORDER by valid DESC
""")
dates = []
winds = []
for row in cursor:
    print(row)
    winds.append( row[1] * 2.23)
    dates.append( row[0] )
    
POSTGIS.close()

In [ ]:
import datetime
dates.append(datetime.date(2013,12,19))

In [2]:
import datetime
import numpy as np
from pyiem.datatypes import temperature
import pyiem.meteorology as meteor
dbconn = get_dbconn('postgis')
cursor = dbconn.cursor()

ar_whght = []
ar_tmpc = []
ar_dwpc = []
for date in wdates:
    if date.year < 1947:
        continue
    cursor.execute("""
    select height, tmpc, dwpc, smps * 2.23 from raob_profile_%s p JOIN raob_flights f on 
     (p.fid = f.fid) where f.station in ('KTIK','KOUN')  and 
     p.height < 7000 and tmpc >= -100 and dwpc >= -100 and 
     valid = '%s 00:00+00'::timestamptz  ORDER by height ASC
    """ % (date.year, (date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")))
    h = []
    t = []
    d = []
    for row in cursor:
        h.append( row[0] )
        t.append( row[1] )
        d.append(row[2])
    if len(h) == 0:
        continue
    print(date, cursor.rowcount, t[0], h[0])
    ar_whght.append(np.array(h))
    ar_tmpc.append(np.array(t))
    ar_dwpc.append(np.array(d))
        

1967-12-15 8 -2.7 387.0
1968-01-02 7 2.1 387.0
1968-01-03 7 -6.4 387.0
1968-01-09 10 -0.6 387.0
1970-01-17 10 -3.9 387.0
1970-03-11 12 -0.6 387.0
1990-02-14 26 -1.1 357.0
1990-03-23 34 2.8 357.0
1991-01-02 40 -1.7 357.0
1991-01-04 38 -2.7 357.0
1991-01-09 49 1.5 357.0
1991-01-10 40 0.8 357.0
1998-01-07 19 0.0 362.0
1998-01-12 21 2.0 362.0
1999-01-07 34 -2.6 362.0
2000-12-25 24 -2.7 362.0
2001-02-15 27 -1.6 362.0
2001-02-27 35 2.2 362.0
2001-02-28 39 -2.9 362.0
2002-03-01 20 6.2 362.0
2002-12-04 15 -2.3 362.0
2002-12-23 16 0.2 362.0
2003-02-15 18 1.6 362.0
2003-02-25 16 -4.5 362.0
2003-02-26 26 -2.1 362.0
2004-02-01 13 5.4 362.0
2006-02-19 20 -3.3 362.0
2007-01-31 32 -1.9 362.0
2007-12-09 35 -1.7 362.0
2007-12-10 53 0.0 362.0
2008-01-21 29 1.8 362.0
2009-01-26 52 -3.1 362.0
2009-03-27 34 2.6 362.0
2010-01-06 23 3.0 362.0
2010-02-01 25 0.0 362.0
2013-12-21 29 -0.1 362.0
2014-03-01 25 5.0 362.0
2015-02-15 33 5.4 362.0
2015-02-28 25 -3.3 362.0
2015-11-27 20 0.6 362.0
2015-12-27 21 0.8 362.

In [20]:
from pyiem.plot import fitbox
def compute_mean(h, s):
    ''' Compute a mean profile '''
    x = []
    y = []
    for hght in range(300,4301,200):
        vals = []
        for myh, mys in zip(h,s):
            for myhh, myss in zip(myh, mys):
                if myhh >= hght and myhh < (hght+200):
                    vals.append(myss)
        x.append( sum(vals) / float(len(vals)))
        y.append( hght + 100)
    return x, y
import matplotlib.pyplot as plt

(fig, ax) = plt.subplots(1,2, sharey=True, dpi=100, figsize=(12, 6.75), facecolor='white')
for i, (h,s) in enumerate(zip(ar_whght,ar_tmpc)):
    if max(s) > 100:
        continue
    # n = np.array(s) / s[0]
    ax[0].plot(s,h, color='tan')

ax[0].plot(ar_tmpc[-1], ar_whght[-1], color='r', lw=2, zorder=2, label='9 Feb 2021')
x, y = compute_mean(ar_whght, ar_tmpc)
ax[0].plot(x ,y, color='k', lw=2, zorder=2, label='Composite')
ax[0].legend(ncol=2, loc=4)
    
for i, (h,s) in enumerate(zip(ar_whght,ar_dwpc)):
    if max(s) > 100:
        continue
    # n = np.array(s) / s[0]
    ax[1].plot(s,h, color='tan')
ax[1].plot(ar_dwpc[-1], ar_whght[-1], color='r', lw=2, zorder=2, label='9 Feb 2021')

x, y = compute_mean(ar_whght, ar_dwpc)
ax[1].plot(x,y, color='k', lw=2, zorder=2, label='Composite')
ax[1].legend(ncol=2, loc=4)
ax[1].grid(True)
ax[0].grid(True)
    
ax[0].set_ylim(0,4000)
ax[0].set_xlim(-20,14)
ax[1].set_xlim(-20,14)
ax[0].set_xlabel(r"Air Temperature $^\circ$C")
ax[1].set_xlabel(r"Dew Point $^\circ$C")
ax[0].set_ylabel("Sounding Height [m]")
fitbox(fig, "00 UTC Norman Sounding on %s Dates of 21-3 UTC KOKC FZDZ Reports" % (len(ar_dwpc),), 0.1, 0.9, 0.9, 0.95)
fig.savefig("test.png")
plt.close()

In [ ]:
fig.savefig('test.png')
